In [22]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta

In [23]:
# Importing and defining a variable for each spreadsheet 


###################################################################
folder_name = '2021_09_set'  # <<< ALTERE O NOME DA PASTA AO LADO #
###################################################################


df1 = pd.read_excel(folder_name + "\Suzlon_Set (1).xlsx")
df2 = pd.read_excel(folder_name + "\Suzlon_Set (2).xlsx")
df3 = pd.read_excel(folder_name + "\Suzlon_Set (3).xlsx")
df4 = pd.read_excel(folder_name + "\Suzlon_Set (4).xlsx")
df5 = pd.read_excel(folder_name + "\Suzlon_Set (5).xlsx")


In [24]:
# Appending all spreadsheets into one and counting the number of rows  

df = df1.append(df2, ignore_index = True)
df = df.append(df3, ignore_index = True)
df = df.append(df4, ignore_index = True)
df = df.append(df5, ignore_index = True)

print('%s is the total number of lines in this spreadsheet' %len(df))

1736 is the total number of lines in this spreadsheet


In [25]:
##Changing columns names.

df.rename(columns = {"respDescription" : "Responsability",
                     "Power Plant" : "Power_plant", 
                     "Element" : "WTG",
                     "Downtime" : "Downtime_type",
                     "Time from" : "Time_from",
                     "Time to" : "Time_to"
                    }, inplace = True)

In [26]:
# Adjusting the values in each column for the right data type.

df["Responsability"] = df["Responsability"].astype("category")
df["Power_plant"] = df["Power_plant"].astype("category")
df["WTG"] = df["WTG"].astype("category")
df["Downtime_type"] = df["Downtime_type"].astype("category")
df["Time_from"] = pd.to_datetime(df["Time_from"])
df["Time_to"] = pd.to_datetime(df["Time_to"])

In [27]:
## changing time before the working month to the first day/hour/second of 
## the month to study only the analysed month.

time = dt.datetime.now()
time_to = time.replace( day=1, hour=0, minute=0, second=0, microsecond=0)
time_from = time_to - relativedelta(months=1)
# print('time from: %s' %time_from)
# print('time to  : %s' %time_to)


mask = df["Time_from"]  <= time_from
df.loc[mask, "Time_from"] = time_from
# df["Time_from"].min()


## changing time before the working month to the first day/hour/second of 
## the month to study only the analysed month.

mask2 = df["Time_to"]  >= time_to
df.loc[mask2, "Time_to"] = time_to
# df["Time_to"].max()

In [28]:
## Converting timedelta "Duration" column to a float number.

df["Duration"] = df["Time_to"] - df["Time_from"]
mask3 = df["Duration"]
df["Duration"] = mask3.dt.total_seconds()/3600

In [29]:
## separating the alarm code column into categories: alarm number, code and description.

df[["Alarm_number", "Alarm_code","Alarm_descrition"]] = df.Alarm.str.split(" - ", expand = True)
df.drop(columns = "Alarm", inplace = True)

In [30]:
## filling na with corrective maintenance code and setting it as integer.

df["Alarm_number"] = df["Alarm_number"].fillna("2")
df["Alarm_number"] = df["Alarm_number"].astype("int")

In [31]:
## opening up the suzlon codes spreadsheet as reference for system and alarm correlation.

suzlon_codes = pd.read_excel("suzlon_codes.xlsx")
# suzlon_codes.info()

In [32]:
## Joining system by alarms in the df.

df = df.merge(suzlon_codes, how = "inner",  left_on = "Alarm_number", right_on = "number" )
df.drop(columns = "number", inplace = True)

In [33]:
## Joining System stops by alarms to the table

suzlon_powerplant = pd.read_excel("suzlon_powerplant.xlsx")
# suzlon_powerplant.info()

df = df.merge(suzlon_powerplant, how = "left",  left_on = "Power_plant", right_on = "Power_plants")
df.drop(columns = "Power_plants", inplace = True)

In [34]:
## adjusting the data types of some columns.

df["Power_plant"] = df["Power_plant"].astype("category")
df["system"] = df["system"].astype("category")
df["Complex"] = df["Complex"].astype("category")
df["Status"] = df["Status"].astype("category")

In [35]:
## organizing the df to the new index.

df.sort_values(["Power_plant","WTG","Time_from"], ascending=[True, True, True], inplace=True)
df.reset_index(drop = True, inplace = True)

In [38]:
## creating the output file with the new dataframe.

file_name = 'complete_list_suzlon_' + time_from.strftime("%m") + '.xlsx'
excel_file = pd.ExcelWriter(file_name)
df.to_excel(excel_file, sheet_name = "Suzlon", index=False, freeze_panes = (1,0))
excel_file.save()